In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import plotly.express as px
import libpysal as lps
from esda.moran import Moran_Local

plt.style.use('seaborn-v0_8')  # Estilo moderno
plt.rcParams['font.family'] = 'Arial'  # Fuente profesional

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from mypackage import dir

# Environment variables
modality = 'u'
project = 'Ciencia de los datos'
data = dir.make_dir_line(modality, project) 
processed = data('processed')
models = data('models')
outputs = data('outputs')

In [ ]:
def cargar_datos(table_name: str) -> pd.DataFrame:
    df = pd.read_parquet(processed / f'{table_name}.parquet.gzip')
    print(f'Loaded table: {table_name}')
    return df

# Función para cargar los datos en la base de datos
def cargar_datos_geo(table_name: str) -> gpd.GeoDataFrame:
    geo = gpd.read_file(processed / f'{table_name}.geojson', encoding='utf-8')
    print(f'Loaded geo table: {table_name}')
    return geo

def create_italy_choropleth(
    df,
    color_column='value',
    location_id='geoid',
    hover_vars=None,  # Dictionary of {column: label} for hover data}
    orden_categorias=None, # List of for order data}
    title="Distribución Poblacional en Italia por Grupos Demográficos",
    source_text="Fuente: XYZ",
    center_lat=41.93309,
    center_lon=12.13129,
    zoom=3.8,
    opacity=0.4,
    map_style='carto-positron',
    color_scale=px.colors.sequential.Blues
):
    """Create an elegant choropleth map of Italy with customizable hover data.
    
    Args:
        df: GeoDataFrame with geographical data and values to plot
        color_column: Column name for color values
        location_id: Column name for location IDs (must match geojson properties)
        hover_vars: Dictionary of {column: label} for hover data display
                   Example: {'DEN_PROV': 'Territorio', 'value': 'Unidades'}
        title: Map title
        source_text: Text for data source annotation
        center_lat: Latitude for map center
        center_lon: Longitude for map center
        zoom: Initial zoom level
        opacity: Opacity of choropleth fill
        map_style: Mapbox style specification
        color_scale: Color scale for the choropleth
    
    Returns:
        A Plotly Figure object
    """
    # Set default hover variables if none provided
    if hover_vars is None:
        hover_vars = {
            'DEN_PROV': 'Territorio',
            color_column: 'Valor'
        }
    
    # Prepare hover data and labels
    hover_data = list(hover_vars.keys())
    hover_labels = {col: label for col, label in hover_vars.items()}
    
    fig = px.choropleth_mapbox(
        df,
        geojson=df.__geo_interface__,
        locations=location_id,
        color=color_column,
        category_orders={color_column: orden_categorias},
        hover_data=hover_data,
        labels=hover_labels,
        featureidkey=f'properties.{location_id}',
        center={'lat': center_lat, 'lon': center_lon},
        mapbox_style=map_style,
        zoom=zoom,
        opacity=opacity,
        title=title,
        # color_continuous_scale=color_scale
    )

    # Generate dynamic hover template
    hover_template_parts = [
        f"<b>{label}</b>: %{{customdata[{i}]}}"
        if not isinstance(df[col].iloc[0], (int, np.int64, float))
        else (
            f"<b>{label}</b>: %{{customdata[{i}]:,}}"
            if df[col].iloc[0] > 1
            # else f"<b>{label}</b>: %{{customdata[{i}]:.3f}}"
            else f"<b>{label}</b>: %{{customdata[{i}]:.2%}}"
        )
        for i, (col, label) in enumerate(hover_vars.items())
    ]

    fig.update_traces(
        hovertemplate="<br>".join(hover_template_parts) + "<extra></extra>",
        marker_line_width=0.5,
        marker_line_color='white'
    )

    # Update layout for better appearance
    fig.update_layout(
        margin={'l': 0, 't': 60, 'r': 0, 'b': 40},
        title={
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 16}
        },
        annotations=[dict(
            text=source_text,
            x=0.5,
            y=-0.1,
            showarrow=False,
            xref="paper",
            yref="paper",
            font={'size': 10}
        )],
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Arial"
        )
    )

    # Update colorbar
    colorbar_title = hover_vars.get(color_column, 'Value')
    fig.update_coloraxes(
        colorbar=dict(
            title=dict(text=colorbar_title, side='right'),
            lenmode="fraction",
            len=0.6,
            yanchor="middle",
            y=0.5,
            x=0,
            thickness=15,
            tickformat=","
        )
    )

    return fig

In [ ]:
orden_categorias = ['HH', 'LH', 'LL', 'HL']

Nota: Se almaceno el area

In [ ]:
geo_provincias = cargar_datos_geo('provincias')
provincias = cargar_datos('provincias')

censo = cargar_datos('censo_tranformado')
censo = censo[censo['marital_status'].isin(['never married'])]
censo = censo[(censo['age'] >= 18) & (censo['age'] <= 28)]

df = pd.merge(censo, provincias, on=['itter107'])

df = df.loc[:,['itter107', 'territory', 'sexistat1', 'value', 'COD_PROV', 'DEN_PROV']]
df = df.groupby(['COD_PROV', 'territory', 'itter107', 'sexistat1'], as_index=False)['value'].sum()

df = pd.merge(geo_provincias, df, on=['COD_PROV'])

df['mt2'] = df['value'] / df['Shape_Area']

df = df.loc[:,['COD_PROV', 'geometry', 'territory', 'itter107', 'sexistat1', 'mt2']]

df_males = df.copy()
df_males = df_males[df_males['sexistat1'] == 1]
df_males = df_males.loc[:,['COD_PROV', 'territory', 'geometry', 'mt2']]

df_females = df.copy()
df_females = df_females[df_females['sexistat1'] == 2]
df_females = df_females.loc[:,['COD_PROV', 'territory', 'geometry', 'mt2']]

In [ ]:
# Crear la matriz de pesos espaciales
w = lps.weights.Queen.from_dataframe(df_males)

# Normalizar la matriz de pesos
w.transform = 'r'

# Calcular el Índice de Moran Local
moran_local = Moran_Local(df_males['mt2'], w)

# Añadir los resultados al GeoDataFrame
df_males['I'] = moran_local.Is
df_males['p_value'] = moran_local.p_sim
df_males['quadrant'] = moran_local.q

# Mapear los valores de quadrant a las categorías HH, HL, LH, LL
df_males['cluster'] = df_males['quadrant'].map({1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'})

# Ver los resultados
df_males.head()

In [ ]:
mapita = create_italy_choropleth(
    df_males,
    color_column='cluster',
    hover_vars={#"DEN_PROV":'Territorio', 
                'territory':'Territorio', 
                'quadrant':'Quadrant', 
                'cluster': 'Grupo',
                #'value':'Unidades', 
                #'proportion': 'Proporción'
                },
    orden_categorias=orden_categorias,
    # value_label='Unidades',
    location_id='COD_PROV',
    title="Autocorrelación espacial de hombres solteros",
    source_text="Fuente: Datos demográficos 2025 | Visualización: Python",
    # center_lat=41.93309,
    # center_lon=12.13129,
    # zoom=3.8,
    # opacity=0.4,
    # map_style='carto-positron'
)
# mapita.write_html(output / 'spatial_autocorrelation_of_single_men.html')
mapita

In [ ]:
# Crear la matriz de pesos espaciales
w = lps.weights.Queen.from_dataframe(df_females)

# Normalizar la matriz de pesos
w.transform = 'r'

# Calcular el Índice de Moran Local
moran_local = Moran_Local(df_females['mt2'], w)

# Añadir los resultados al GeoDataFrame
df_females['I'] = moran_local.Is
df_females['p_value'] = moran_local.p_sim
df_females['quadrant'] = moran_local.q

# Mapear los valores de quadrant a las categorías HH, HL, LH, LL
df_females['cluster'] = df_females['quadrant'].map({1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'})
# Ver los resultados
df_females.head()

In [ ]:
mapita = create_italy_choropleth(
    df_females,
    color_column='cluster',
    hover_vars={#"DEN_PROV":'Territorio', 
                'territory':'Territorio', 
                'quadrant':'Quadrant', 
                'cluster': 'Grupo',
                #'value':'Unidades', 
                #'proportion': 'Proporción'
                },
    orden_categorias=orden_categorias,
    # value_label='Unidades',
    location_id='COD_PROV',
    title="Autocorrelación espacial de mujeres solteras",
    source_text="Fuente: Datos demográficos 2025 | Visualización: Python",
    # center_lat=41.93309,
    # center_lon=12.13129,
    # zoom=3.8,
    # opacity=0.4,
    # map_style='carto-positron'
)
# mapita.write_html(output / 'spatial_autocorrelation_of_single_women.html')
mapita

In [ ]:
# Crear la figura con 2 subplots horizontales (1 fila, 2 columnas)
fig, axes = plt.subplots(1, 2, figsize=(20, 10), facecolor='white')
fig.suptitle('Spatial Autocorrelation of Single', 
             fontsize=20, y=1.05, fontweight='bold')

# Suponiendo que tienes 3 DataFrames diferentes (df1, df2, df3) para cada mapa
# Si es el mismo DataFrame con diferentes columnas, ajusta según corresponda

# Lista de DataFrames y títulos
dfs = [df_males, df_females]
titles = ['Males', 
          'Females']
cmap = 'coolwarm'


# Crear cada submapa
for ax, df, title in zip(axes, dfs, titles):
    # Mapa principal
    plot = df.plot(ax=ax, column='cluster', cmap=cmap, 
                   edgecolor='white', linewidth=0.3)
    
    # Título
    ax.set_title(title, fontsize=16, pad=15, fontweight='semibold')
    
    # Remover ejes
    ax.set_axis_off()
    
    # Barra de color profesional
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    sm = plt.cm.ScalarMappable(cmap=cmap)
    sm._A = []
    fig.colorbar(sm, cax=cax, orientation='vertical')
    cax.set_ylabel('Proportion', rotation=90, labelpad=15, fontsize=12)

# Ajustes finales
plt.tight_layout(pad=3.0)  # Espaciado entre subplots

# # Añadir anotaciones
# plt.annotate('Fuente: Instituto Nacional de Estadística de Italia | Elaboración propia', 
#              xy=(0.5, 0), 
#              xytext=(0, -30),
#              xycoords='figure points', textcoords='offset points',
#              ha='center', va='top', fontsize=12, color='gray'
#              )

# Guardar en alta calidad
# plt.savefig('spatial_autocorrelation_of_single.png', dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())

# Mostrar el gráfico
plt.show()